In [87]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#전처리
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer

#데이터분리
from sklearn.model_selection import StratifiedKFold, KFold, GridSearchCV, train_test_split

#모델링
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from lightgbm import LGBMClassifier
import warnings
warnings.filterwarnings('ignore')
#성과 평가 지표
from sklearn.metrics import accuracy_score, mean_squared_error

# auto-PyTorch

import os
import tempfile as tmp
import warnings

os.environ['JOBLIB_TEMP_FOLDER'] = tmp.gettempdir()
os.environ['OMP_NUM_THREADS'] = '1'
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'

warnings.simplefilter(action='ignore', category=UserWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

import sklearn.datasets
import sklearn.model_selection

import autoPyTorch
from autoPyTorch.api.tabular_classification import TabularClassificationTask


In [88]:
pwd

'/Users/anseon-yeong/psychological_tendency/challange/code'

In [89]:
train=pd.read_csv('../data/train.csv', index_col=0)
test=pd.read_csv('../data/test.csv', index_col=0)
submission=pd.read_csv('../data/sample_submission.csv', index_col=0)  
print(train.shape)
print(test.shape)
print(submission.shape)

(15000, 69)
(35452, 68)
(35452, 1)


In [90]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15000 entries, 0 to 14999
Data columns (total 69 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Q1            14959 non-null  float64
 1   Q2            14931 non-null  float64
 2   Q3            14950 non-null  float64
 3   Q4            14929 non-null  float64
 4   Q5            14962 non-null  float64
 5   Q6            14952 non-null  float64
 6   Q7            14924 non-null  float64
 7   Q8            14952 non-null  float64
 8   Q9            14944 non-null  float64
 9   Q10           14928 non-null  float64
 10  Q11           14941 non-null  float64
 11  Q12           14933 non-null  float64
 12  Q13           14960 non-null  float64
 13  Q14           14964 non-null  float64
 14  Q15           14955 non-null  float64
 15  Q16           14967 non-null  float64
 16  Q17           14963 non-null  float64
 17  Q18           14937 non-null  float64
 18  Q19           14947 non-nu

In [91]:
#labelencoding
label = LabelEncoder()

train['gender_code'] = label.fit_transform(train['gender'])
train['religion_code'] = label.fit_transform(train['religion'])

test['gender_code'] = label.fit_transform(test['gender'])
test['religion_code'] = label.fit_transform(test['religion'])

drop2 = ['country','gender','religion']
train = train.drop(drop2, axis = 1)
test = test.drop(drop2, axis = 1)

In [92]:
#이상치를 NaN으로 변경 
train.mask(train.sub(train.mean()).div(train.std()).abs().gt(2))
test.mask(test.sub(test.mean()).div(test.std()).abs().gt(2))

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,engnat,age,hand,orientation,voted,married,familysize,ASD,gender_code,religion_code
index,,,,,,,,,,,,,,,,,,,,,
0,4.0,4.0,3.0,5.0,5.0,5.0,3.0,5.0,4.0,5.0,...,2.0,19.0,1.0,4.0,1.0,1.0,3.0,2.0,1.0,3.0
1,4.0,5.0,4.0,4.0,5.0,4.0,5.0,5.0,5.0,4.0,...,1.0,33.0,1.0,NaN,2.0,1.0,5.0,2.0,0.0,0.0
2,5.0,5.0,5.0,5.0,4.0,5.0,5.0,5.0,5.0,4.0,...,2.0,13.0,1.0,NaN,2.0,1.0,3.0,2.0,1.0,3.0
3,5.0,4.0,3.0,4.0,5.0,4.0,5.0,4.0,4.0,5.0,...,1.0,28.0,1.0,2.0,2.0,1.0,3.0,2.0,0.0,1.0
4,5.0,5.0,5.0,5.0,5.0,3.0,5.0,5.0,5.0,5.0,...,2.0,15.0,1.0,NaN,2.0,1.0,2.0,2.0,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35447,4.0,5.0,5.0,3.0,3.0,4.0,5.0,5.0,4.0,3.0,...,2.0,16.0,NaN,4.0,2.0,1.0,3.0,2.0,1.0,9.0
35448,5.0,5.0,5.0,5.0,5.0,4.0,5.0,3.0,5.0,3.0,...,1.0,16.0,NaN,NaN,2.0,1.0,2.0,2.0,NaN,1.0
35449,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,1.0,31.0,1.0,NaN,1.0,1.0,2.0,2.0,0.0,0.0


In [93]:
#missing 확인
print('train : ',train.isnull().sum())
print('test : ',test.isnull().sum())

train :  Q1                41
Q2                69
Q3                50
Q4                71
Q5                38
                ... 
familysize       319
ASD               89
nerdiness          0
gender_code        0
religion_code      0
Length: 68, dtype: int64
test :  Q1               119
Q2               147
Q3               130
Q4               125
Q5                90
                ... 
married          180
familysize       703
ASD              237
gender_code        0
religion_code      0
Length: 67, dtype: int64


In [94]:
#결측치 보간 
imp_mean = SimpleImputer(missing_values=np.nan,strategy='mean')
train = pd.DataFrame(imp_mean.fit_transform(train), columns=train.columns)
test = pd.DataFrame(imp_mean.fit_transform(test), columns=test.columns)
test

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,engnat,age,hand,orientation,voted,married,familysize,ASD,gender_code,religion_code
0,4.0,4.0,3.0,5.0,5.0,5.0,3.0,5.0,4.0,5.0,...,2.0,19.0,1.0,4.0,1.0,1.0,3.000000,2.0,1.0,3.0
1,4.0,5.0,4.0,4.0,5.0,4.0,5.0,5.0,5.0,4.0,...,1.0,33.0,1.0,5.0,2.0,1.0,5.000000,2.0,0.0,0.0
2,5.0,5.0,5.0,5.0,4.0,5.0,5.0,5.0,5.0,4.0,...,2.0,13.0,1.0,5.0,2.0,1.0,3.000000,2.0,1.0,3.0
3,5.0,4.0,3.0,4.0,5.0,4.0,5.0,4.0,4.0,5.0,...,1.0,28.0,1.0,2.0,2.0,1.0,3.000000,2.0,0.0,1.0
4,5.0,5.0,5.0,5.0,5.0,3.0,5.0,5.0,5.0,5.0,...,2.0,15.0,1.0,5.0,2.0,1.0,2.000000,2.0,1.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35447,4.0,5.0,5.0,3.0,3.0,4.0,5.0,5.0,4.0,3.0,...,2.0,16.0,3.0,4.0,2.0,1.0,3.000000,2.0,1.0,9.0
35448,5.0,5.0,5.0,5.0,5.0,4.0,5.0,3.0,5.0,3.0,...,1.0,16.0,3.0,5.0,2.0,1.0,2.000000,2.0,2.0,1.0
35449,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,...,1.0,31.0,1.0,5.0,1.0,1.0,2.000000,2.0,0.0,0.0
35450,5.0,5.0,4.0,5.0,5.0,1.0,5.0,1.0,5.0,5.0,...,1.0,19.0,1.0,1.0,2.0,1.0,2301.835621,2.0,0.0,11.0


In [96]:
# Data Loading
# ============
X_train = train.loc[:,'Q1':'ASD']
Y_train = train['nerdiness']
X_train.info

<bound method DataFrame.info of         Q1   Q2   Q3   Q4   Q5   Q6   Q7   Q8   Q9  Q10  ...  education  \
0      1.0  5.0  5.0  5.0  1.0  4.0  5.0  5.0  1.0  3.0  ...        2.0   
1      4.0  4.0  4.0  4.0  4.0  5.0  4.0  4.0  3.0  3.0  ...        4.0   
2      4.0  5.0  5.0  4.0  3.0  5.0  5.0  5.0  4.0  4.0  ...        2.0   
3      4.0  4.0  4.0  2.0  4.0  3.0  3.0  5.0  3.0  4.0  ...        1.0   
4      4.0  4.0  4.0  4.0  3.0  3.0  4.0  2.0  3.0  4.0  ...        1.0   
...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...        ...   
14995  2.0  5.0  4.0  3.0  3.0  4.0  4.0  4.0  3.0  4.0  ...        2.0   
14996  5.0  4.0  5.0  4.0  4.0  5.0  5.0  4.0  4.0  5.0  ...        4.0   
14997  4.0  5.0  5.0  5.0  5.0  5.0  5.0  5.0  4.0  5.0  ...        2.0   
14998  5.0  5.0  4.0  5.0  5.0  5.0  5.0  1.0  5.0  5.0  ...        3.0   
14999  5.0  4.0  2.0  5.0  2.0  2.0  4.0  2.0  4.0  4.0  ...        2.0   

       urban  engnat   age  hand  orientation  voted  married  fami

In [23]:
pwd

'/Users/anseon-yeong/psychological_tendency/challange/code'

In [97]:
# Build and fit a classifier
# ==========================
api = TabularClassificationTask(
    # To maintain logs of the run, you can uncomment the
    # Following lines
    #temporary_directory='./tmp/autoPyTorch_example_tmp_01',
    # output_directory='./tmp/autoPyTorch_example_out_01',
    # delete_tmp_folder_after_terminate=False,
    # delete_output_folder_after_terminate=False,
    seed=57,
)

In [56]:
os.environ['JOBLIB_TEMP_FOLDER'] = tmp.gettempdir()
os.environ['OMP_NUM_THREADS'] = '10'
os.environ['OPENBLAS_NUM_THREADS'] = '10'
os.environ['MKL_NUM_THREADS'] = '10'
cls = TabularClassificationTask(n_jobs=10)

In [58]:
############################################################################
# Search for an ensemble of machine learning algorithms
# =====================================================
cls.search(
    X_train=X_train,
    y_train=Y_train,
    #dataset_name='Australian',
    optimize_metric='roc_auc',
    total_walltime_limit=300,
    func_eval_time_limit_secs=50
)

In [98]:
############################################################################
# Search for an ensemble of machine learning algorithms
# =====================================================
api.search(
    X_train=X_train,
    y_train=Y_train,
    #dataset_name='Australian',
    optimize_metric='roc_auc',
    total_walltime_limit=300,
    func_eval_time_limit_secs=50
)

[ERROR] [2022-08-15 19:32:09,363:Client-AutoPyTorch:4ed6f22a-1c85-11ed-b85d-acde48001122:57] Traditional prediction for svm failed with run state StatusType.TIMEOUT.
Additional info:
error: Timeout
configuration_origin: traditional


In [99]:
y_pred2 = api.predict(X_test)

In [100]:
print(api.show_models())


|    | Preprocessing   | Estimator   |   Weight |
|---:|:----------------|:------------|---------:|
|  0 | None            | ETLearner   |     0.5  |
|  1 | None            | CBLearner   |     0.24 |
|  2 | None            | LGBMLearner |     0.18 |
|  3 | None            | RFLearner   |     0.06 |
|  4 | None            | KNNLearner  |     0.02 |


In [60]:
#y_pred = cls.predict(X_test)

In [48]:
# Print the final ensemble performance
# ====================================
#y_pred = api.predict(x_valid)
score = api.score(y_pred, y_valid)
print(score)
# Print the final ensemble built by AutoPyTorch
print(api.show_models())

# Print statistics from search
print(api.sprint_statistics())


{'roc_auc': 0.6814386660097144}
|    | Preprocessing                                                                                | Estimator                                                          |   Weight |
|---:|:---------------------------------------------------------------------------------------------|:-------------------------------------------------------------------|---------:|
|  0 | SimpleImputer,Variance Threshold,NoCoalescer,NoEncoder,StandardScaler,NoFeaturePreprocessing | no embedding,ShapedMLPBackbone,FullyConnectedHead,nn.Sequential    |     0.84 |
|  1 | None                                                                                         | ETLearner                                                          |     0.08 |
|  2 | None                                                                                         | RFLearner                                                          |     0.04 |
|  3 | SimpleImputer,Variance Threshold,NoCoalescer,NoEnco

In [61]:
y_pred

array([1., 1., 1., ..., 1., 1., 1.])

In [53]:
submission

,nerdiness
index,
0,-1
1,-1
2,-1
3,-1
4,-1
...,...
35447,-1
35448,-1
35449,-1


In [62]:
submission['nerdiness'] = y_pred

In [101]:
submission['nerdiness'] = y_pred2

In [63]:
pwd

'/Users/anseon-yeong/psychological_tendency/challange/code'

In [102]:
submission.to_csv('../auto-py3.csv')